In [1]:
import gzip
import shutil
import os 
import pandas as pd
import numpy as np
import matplotlib as plt

In [2]:
df=pd.read_csv(os.path.join(dir,'GSM5075659_p007n.tsv'), sep='\t')
print(df)

             gene  p007n:AACCCAACAGTTGCGC  p007n:AACAACCGTGTTCAGT  \
0       LINC00115                       0                       0   
1          FAM41C                       0                       0   
2      AL645608.1                       0                       0   
3           NOC2L                       0                       0   
4           PERM1                       0                       0   
...           ...                     ...                     ...   
21849       DDX3Y                       0                       0   
21850      TMSB4Y                       0                       0   
21851      NLGN4Y                       0                       0   
21852      EIF1AY                       0                       1   
21853      RPS4Y2                       0                       0   

       p007n:AACGTCACAACCAACT  p007n:AAACGCTAGACCAAGC  p007n:AACGAAACACGATAGG  \
0                           0                       0                       0   
1        

In [3]:
df.head()

,gene,p007n:AACCCAACAGTTGCGC,p007n:AACAACCGTGTTCAGT,p007n:AACGTCACAACCAACT,p007n:AAACGCTAGACCAAGC,p007n:AACGAAACACGATAGG,p007n:AACCTGATCGGCTGGT,p007n:AAAGGTAGTGTTGAGG,p007n:AACAAGAGTAGTTCCA,p007n:AAACGCTTCGTAGGAG,...,p007n:GTTCATTGTGGCCTCA,p007n:GTTCATTCAACCCTCT,p007n:TACCTCGAGCTGCCAC,p007n:TATCTGTCAAGCGCAA,p007n:TAGACCACAAGAGTAT,p007n:TATGTTCAGTGGTCAG,p007n:TCCTAATTCCGTGTGG,p007n:TGGTACATCTGCGGCA,p007n:TGGCGTGAGACTCAAA,p007n:TTTAGTCCAAGTATAG
0,LINC00115,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,FAM41C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AL645608.1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,NOC2L,0,0,0,0,0,0,2,0,0,...,0,1,0,1,0,0,0,0,0,0
4,PERM1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
df['gene']

0         LINC00115
1            FAM41C
2        AL645608.1
3             NOC2L
4             PERM1
            ...    
21849         DDX3Y
21850        TMSB4Y
21851        NLGN4Y
21852        EIF1AY
21853        RPS4Y2
Name: gene, Length: 21854, dtype: object

In [22]:
for i in df['gene']:
   print(i)

LINC00115
FAM41C
AL645608.1
NOC2L
PERM1
AL645608.8
HES4
RNF223
C1orf159
TTLL10-AS1
TNFRSF18
TNFRSF4
SDF4
C1QTNF12
AL162741.1
UBE2J2
ACAP3
INTS11
DVL1
MXRA8
AURKAIP1
CCNL2
MRPL20
ANKRD65
LINC01770
SSU72
FNDC10
AL691432.2
CDK11B
SLC35E2B
CDK11A
SLC35E2A
NADK
GNB1
TMEM52
CFAP74
AL391845.2
AL391845.1
AL590822.2
PRKCZ-AS1
FAAP20
MORN1
PEX10
PANK4
HES5
TNFRSF14-AS1
MMEL1
TTC34
LINC00982
AL590438.1
MEGF6
AL513320.1
WRAP73
AL136528.1
LRRC47
CEP104
C1orf174
AL805961.2
NPHP4
CHD5
RPL22
AL031847.1
ICMT
GPR153
ACOT7
HES2
TNFRSF25
PLEKHG5
NOL9
KLHL21
DNAJC11
CAMTA1-DT
AL359881.3
AL359881.2
AL359881.1
Z98884.1
UTS2
TNFRSF9
AL034417.3
ERRFI1
RERE
ENO1
SLC2A5
GPR157
MIR34AHG
AL928921.2
PIK3CD-AS1
PIK3CD-AS2
CLSTN1
CTNNBIP1
LZIC
AL139424.3
AL139424.1
DFFA
CASZ1
C1orf127
MASP2
SRM
EXOSC10
MTOR
FBXO2
MAD2L2
MTHFR
KIAA2013
DHRS3
AC254633.1
AL357873.1
TMEM51-AS1
AL031283.2
AL031283.1
CASP9
AL121992.3
AGMAT
AL121992.1
AL121992.2
SLC25A34-AS1
UQCRHL
AL450998.2
ZBTB17
HSPB7
FAM131C
EPHA2
ARHGEF19
CPLANE2
FBXO

PIFO
ATP5PB
C1orf162
RAP1A
DDX20
LINC01750
CTTNBP2NL
WNT2B
CAPZA1
MOV10
AL603832.2
AL390729.1
SLC16A1-AS1
LRIG2
MAGI3
AL137856.1
AP4B1-AS1
DCLRE1B
HIPK1
OLFML3
LINC01765
VANGL1
SLC22A15
MAB21L3
ATP1A1
CD2
PTGFRN
CD101
TTF2
MAN1A2
TENT5C
WDR3
WARS2-IT1
AL359915.2
HSD3B2
HSD3B1
PHGDH
NBPF26
LINC00623
FCGR1B
SRGAP2C
AC244021.1
AC239800.2
FAM72D
AC245014.3
GPR89A
CD160
POLR3C
NUDT17
POLR3GL
LINC01719
AC239799.2
NBPF12
AC242426.2
CHD1L
BCL9
GPR89B
AC239803.2
AC239803.3
PDE4DIP
AC245297.3
AC245297.2
NBPF19
FCGR1A
HIST2H4A
AC239868.3
HIST2H2AA4
HIST2H2AC
BOLA1
VPS45
PLEKHO1
CA14
C1orf54
CIART
MRPS21
PRPF3
RPRD2
TARS2
ECM1
FALEC
ADAMTSL4
SETDB1
AL590133.1
ANXA9
PRUNE1
BNIPL
C1orf56
MLLT11
GABPB2
SCNM1
PIP5K1A
PSMD4
ZNF687
AL391069.4
AL391069.3
PSMB4
CGN
TUFT1
SNX27
AL589765.1
RIIAD1
AL589765.7
OAZ3
TDRKH-AS1
AL589765.6
AL450992.2
AL450992.1
LCE2A
LCE1E
SPRR5
SPRR1A
SPRR3
SPRR1B
LOR
S100A9
S100A7A
BX470102.1
S100A1
CHTOP
SNAPIN
NPR1
INTS3
SLC27A3
AL358472.5
AL358472.4
AL358472.3
CREB3L4
AL35847

TNKS1BP1
SSRP1
SLC43A3
SLC43A1
TIMM10
UBE2L6
YPEL4
MED19
BTBD18
AP003557.1
OR5B21
LPXN
AP001350.1
AP001636.3
FAM111A-DT
MPEG1
AP002358.1
AP000442.1
OSBP
PATL1
AP000640.1
MRPL16
GIF
TCN1
AP001257.1
MS4A6A
MS4A4E
PTGDR2
PRPF19
AP003721.3
AP003721.1
SLC15A3
VPS37C
VWCE
DDB1
CYB561A3
CPSF7
SYT7
TMEM258
FADS1
FADS3
RAB3IL1
FTH1
AP003733.3
AP003306.2
AHNAK
EEF1G
TUT1
MTA2
EML3
B3GAT3
GANAB
INTS5
C11orf98
LBHD1
UBXN1
LRRN4CL
BSCL2
HNRNPUL2-BSCL2
HNRNPUL2
ZBTB3
AP001160.2
TMEM223
NXF1
STX5
WDR74
CHRM1
HRASLS5
HRASLS2
PLA2G16
ATL3
C11orf95
RCOR2
MACROD1
TRPT1
AP001453.1
PPP1R14B
AP001453.4
BAD
TRMT112
AP003774.4
AP003774.3
NRXN2
RASGRP2
PYGM
SF1
MAP4K2
MEN1
CDC42BPG
EHD1
MIR194-2HG
ATG2A
BATF2
NAALADL1
CDCA5
TMEM262
AP003068.1
ZNHIT2
FAU
SYVN1
AP003068.2
SLC25A45
AP000769.3
AP000769.2
LTBP3
KCNK7
MAP3K11
RELA
RNASEH2C
AP5B1
OVOL1-AS1
AP001266.1
CFL1
EFEMP2
FIBP
FOSL1
C11orf68
EIF1AD
CATSPER1
AP006287.2
AP000759.1
AP001107.4
AP001107.1
YIF1A
CD248
RIN1
BRMS1
B4GAT1
SLC29A2
MRPL11
AP002748.3
ZDHH

ETFRF1
AC092794.1
AC092794.2
RASSF8
SSPN
FGFR1OP2
AC024896.1
MED21
STK38L
ARNTL2
SMCO2
PPFIBP1
REP15
AC009509.4
AC009509.2
MRPS35
KLHL42
AC008011.2
CCDC91
FAR2
OVCH1-AS1
AC010198.1
LINC00941
TSPAN11
DDX11
AC024940.1
ETFBKMT
AC023157.3
KIAA1551
AC016957.2
BICD1
FGD4
DNM1L
ALG10
ALG10B
AC067735.1
LRRK2
CNTN1
PDZRN4
PPHLN1
AC079601.1
IRAK4
TMEM117
ANO6
ARID2
AC025031.2
AC008014.1
AC008035.1
AC079906.1
PCED1B
AC008083.2
AC008083.3
LINC02416
LINC02156
AC004241.1
SLC48A1
AC004466.1
LINC02354
AC121338.1
AC121338.2
TMEM106C
PFKM
AC074029.3
CCDC184
AC024257.3
AC117498.1
AC117498.2
CACNB3
CCDC65
DDN-AS1
AC011603.2
TUBA1C
PRPH
TROAP
DNAJC22
SPATS2
KCNH3
PRPF40B
TMBIM6
BCDIN3D-AS1
AQP5
AQP6
ASIC1
SMARCD1
GPD1
COX14
AC074032.1
AC008147.1
LARP4
DIP2B
ATF1
METTL7A
LETMD1
DAZAP2
SLC4A8
SCN8A
ANKRD33
ACVRL1
ACVR1B
GRASP
NR4A1
ATG101
AC078864.2
LINC00592
KRT7
KRT86
KRT18
EIF4B
TNS2
IGFBP6
ZNF740
AC021072.1
MFSD5
ESPL1
PFDN5
C12orf10
SP1
AMHR2
PRR13
PCBP2
TARBP2
AC023509.2
AC076968.1
HOXC11
HOXC9
HOXC8
H

C14orf180
AL583722.1
INF2
ADSSL1
SIVA1
AL583722.2
ZBTB42
LINC00638
AL583810.2
CEP170B
AL583810.3
PLD4
CLBA1
LINC02298
BTBD6
PACS2
TEX22
MTA1
CRIP2
CRIP1
TEDC1
TMEM121
AC246787.2
FAM30A
LINC00226
IGHV3OR15-7
AC087386.1
AC012414.5
CYFIP1
TUBGCP5
GOLGA6L2
MAGEL2
NDN
UBE3A
ATP10A
GABRB3
OCA2
HERC2
GOLGA8M
GOLGA6L7
FAM189A1
TJP1
AC026150.3
AC091057.2
MTMR10
AC012236.1
OTUD7A
AC123768.3
AC090877.2
FMN1
AC010809.3
AC010809.2
AC010809.1
AVEN
EMC7
KATNBL1
SLC12A6
NOP10
LPCAT4
GOLGA8A
GOLGA8B
ACTC1
AQR
ZNF770
DPH6
MEIS2
LINC01852
AC087473.1
RASGRP1
AC022929.2
AC013652.1
FSIP1
GPR176
SRP14
BMF
PLCB2
CCDC9B
CCDC32
RMDN3
DNAJC17
PPP1R14D
SPINT1-AS1
RHOV
INO80
EXD1
OIP5
NDUFAF1
LTK
RPAP1
AC073657.1
SPTBN5
EHD4
PLA2G4D
PLA2G4F
VPS39
TMEM87A
ZNF106
AC018362.1
LRRC57
AC018362.2
CDAN1
TTBK2
UBR1
LCMT2
ZSCAN29
TP53BP1
PPIP5K1
STRC
CATSPER2
ELL3
SERINC4
MFAP1
FRMD5
AC025043.1
AC025430.1
EIF3J-DT
SPG11
PATL2
AC122108.2
DUOX2
DUOXA1
SHF
AC051619.5
GATM
AC025580.1
SLC30A4
AC090527.3
MYEF2
AC023355.1
FBN1
CEP

...

TOMM70
ABI3BP
IMPG2
SENP7
ZBTB11
RPL24
AC020651.1
CBLB
LINC00882
CD47
IFT57
MYH15
CIP2A
RETNLB
MORC1
DPPA4
NECTIN3-AS1
ZBED2
GCSAM
SLC9C1
AC092894.1
BTLA
ATG3
CCDC80
LINC02042
CD200R1
AC074044.1
NEPRO
CFAP44
SPICE1
SIDT1-AS1
USF3
NAA50
AC108693.2
AC128687.2
CCDC191
ZNF80
ZBTB20
LSAMP
AC092691.1
IGSF11
B4GALT4
ARHGAP31-AS1
TMEM39A
AC073352.1
CD80
AC073352.2
COX17
GSK3B
LRRC58
AC063952.4
FSTL1
HGD
RABL3
POLQ
HCLS1
GOLGB1
IQCB1
ILDR1
CCDC58
WDR5B
KPNA1
PARP9
HSPBAP1
SEMA5B
ADCY5
HACD2
MYLK
CCDC14
ITGB5
MUC13
HEG1
SLC12A8
ZNF148
SNX4
OSBPL11
AC092903.2
ALG1L
SLC41A3
ALDH1L1
KLF15
ZXDC
TXNRD3
AC084035.1
TPRA1
MGLL
RUVBL1
GATA2
LINC01565
RPN1
AC112484.3
KIAA1257
ISY1
AC108673.3
CNBP
H1FX
EFCAB12
MBD4
PLXND1
TMCC1
LINC02014
PIK3R4
ASTE1
MRPL3
ACAD11
NPHP3
BFSP2-AS1
AC022296.3
TOPBP1
RAB6B
C3orf36
SLCO2A1
LINC02000
RYK
AC010207.1
AMOTL2
ANAPC13
AC072039.2
MSL2
STAG1
AC096992.2
NCK1-DT
DZIP1L
DBR1
NME9
CEP70
PIK3CB
AC024933.1
COPB2
RBP2
RBP1
NMNAT3
ATP1B3-AS1
TFDP2
GK5
XRN1
ATR
PLS1-AS1
PCOLCE2

NFKB1
AC018797.2
CISD2
LINC02428
AC105460.2
TET2
ARHGEF38
GSTCD
NPNT
AIMP1
SGMS2
CYP2U1
HADH
RPL34
OSTC
SEC24B
MCUB
AC004067.1
GAR1
RRH
LRIT3
EGF
ENPEP
FAM241A
AP1AR
ALPK1
AC023886.1
LARP7
ANK2
AC104779.1
UGT8
MTRNR2L13
LINC02263
NDST3
SNHG8
METTL14
SYNPO2
USP53
AC093752.3
AC093752.2
EXOSC9
AC079341.2
KIAA1109
IL21-AS1
BBS12
FGF2
SPATA5
SPRY1
LINC01091
FAT4
INTU
HSPA4L
PLK4
ABHD18
LARP1B
AC108062.1
JADE1
C4orf33
PCDH10
LINC02172
NOCT
NAA15
RAB33B
AC112236.2
MGST2
SCOC
ELMOD2
AC096733.2
ZNF330
IL15
AC139720.1
USP38
GAB1
SMARCA5
HHIP
ABCE1
SMAD1
MMAA
LSM6
REELD1
EDNRA
TMEM184C
ARHGAP10
AC108935.1
LINC02355
DCLK2
AC110813.1
MAB21L2
RPS3A
AC095055.1
PRSS48
FAM160A1
AC092611.2
AC092611.1
AC097375.1
LINC02273
AC106882.1
ARFIP1
FHDC1
TRIM2
MND1
TMEM131L
TLR2
FGB
LRAT
GUCY1A1
GUCY1B1
TDO2
GLRB
FAM198B-AS1
TMEM144
RXFP1
ETFDH
FNIP2
RAPGEF2
NPY5R
TMA16
AC093788.1
SMIM31
APELA
FAM218A
KLHL2
MSMO1
CPE
TLL1
ANXA10
PALLD
AC021151.1
AC096741.1
CLCN3
LINC01612
LINC02512
GALNTL6
GALNT7
SAP30
HAND2-AS1


CENPQ
C6orf141
IL17A
EFHC1
TRAM2-AS1
TMEM14A
FBXO9
AL034374.1
LINC01564
LRRC1
MLIP
TINAG
FAM83B
AL512422.1
BEND6
KIAA1586
ZNF451
BAG2
PRIM2
FKBP1C
PTP4A1
PHF3
AL354719.2
ADGRB3
COL19A1
AL365226.2
FAM135A
SDHAF4
AL583856.2
SMAP1
OGFRL1
KCNQ5
AC019205.1
DDX43
MTO1
AL121972.1
CD109
AL080250.1
SENP6
MYO6
IRAK1BP1
HMGN3-AS1
AL451064.1
SH3BGRL2
TTK
BCKDHB
AL359715.3
AL359715.1
AL359715.2
TPBG
DOPEY1
RWDD2A
PRSS35
RIPPLY2
CYB5R4
MRAP2
NT5E
ZNF292
SMIM8
SLC35A1
ORC3
PNRC1
PM20D2
ANKRD6
CASP8AP2
AL121787.1
AL132996.1
MANEA
UFL1
FHL5
KLHL32
POU3F2
AL513550.1
TSTD3
AL137784.2
PRDM13
AL133338.1
GRIK2
LIN28B
PRDM1
CRYBG1
QRSL1
C6orf203
SOBP
AL024507.2
AFG1L
FOXO3
LINC00222
ARMC2
AL390208.1
CEP57L1
AL359711.2
SMPD2
FIG4
CDC40
AMD1
GTF3C6
RPF2
SLC16A10
MFSD4B
AL080317.1
AL080317.2
AL080317.3
TRAF3IP2-AS1
Z97989.1
WISP3
FAM229B
AL513123.1
MARCKS
FO393415.1
HDAC2-AS2
NT5DC1
DSE
CALHM6
CALHM5
RWDD1
RSPH4A
KPNA5
RFX6
DCBLD1
NUS1
SLC35F1
PLN
ASF1A
GJA1
HSF2
PKIB
SMPDL3A
NKAIN2
RNF217
TPD52L1
HEY2
NCOA7
HI

THAP1
RNF170
CEBPD
PRKDC
AC026904.2
AC026904.3
SNAI2
PXDNL
AC090186.1
PCMTD1
ST18
ALKAL1
RB1CC1
AC022034.2
ATP6V1H
TCEA1
AC100821.2
LYPLA1
TMEM68
RPS20
PLAG1
SDR16C5
IMPAD1
NSMAF
TOX
CA8
LINC01301
AC068389.4
AC068389.1
AC068389.2
AC022182.2
ASPH
AC091173.1
GGH
TTPA
YTHDF3-AS1
AC090136.3
CYP7B1
AC100814.1
ARMC1
AC055822.1
PDE7A
RRS1-AS1
MYBL1
VCPIP1
TCF24
COPS5
ARFGEF1
CPA6
SLCO5A1
NCOA2
TRAM1
LACTB2
EYA1
MSC
TRPA1
SBSPON
C8orf89
RPL7
RDH10-AS1
STAU2
UBE2W
ELOC
JPH1
CASC9
PEX2
IL7
HEY1
AC036214.1
MRPS28
AC036214.3
TPD52
AC009812.1
AC009812.3
ZNF704
PAG1
AC009902.3
AC009902.2
PMP2
FABP4
IMPA1
SLC10A5
ZFAND1
SNX16
AC011773.4
AC011773.1
C8orf59
AC011773.3
CA1
CA3-AS1
RMDN1
CNGB3
MMP16
AC090578.1
AF117829.1
NBN
CALB1
TMEM64
C8orf88
PIP4P2
OTUD6B-AS1
RUNX1T1
TRIQK
C8orf87
LINC00535
AC120053.1
RBM12B
AC084346.2
AP003351.1
CDH17
GEM
RAD54B
VIRMA
AC108860.2
CCNE2
TP53INP1
MIR3150BHG
AC068189.1
C8orf37
GDF6
UQCRB
MTERF3
TSPYL5
RPL30
RIDA
NIPAL2
STK3
AC104986.2
COX6C
RGS22
FBXO43
RNF19A
AP003472.

APEX2
PAGE5
MAGEH1
RRAGB
AL445472.1
KLF8
UBQLN2
AL354793.1
NBDY
AL022157.1
FAAH2
ZXDB
ZC3H12B
MSN
AL034397.3
HEPH
AR
YIPF6
STARD8
EFNB1
FAM155B
EDA
IGBP1
ARR3
RAB41
KIF4A
GDPD2
DLG3
FOXO4
MED12
NLGN3
GJB1
NONO
ITGB1BP2
TAF1
OGT
GCNA
BX276092.9
LINC00891
NHSL2
PIN4
CHIC1
TSIX
JPX
SLC16A2
UPRT
PBDC1
MAGEE1
COX7B
ATP7A
PGK1
LPAR4
P2RY10
GPR174
SH3BGRL
APOOL
ZNF711
DACH2
KLHL4
PABPC5
FAM133A
DIAPH2
TNMD
SRPX2
CSTF2
TMEM35A
CENPI
DRP2
RPL36A
HNRNPH2
ARMCX4
ARMCX1
ARMCX3
TCEAL2
ARMCX5
GPRASP1
GPRASP2
BHLHB9
LINC00630
Z68871.1
RAB40AL
BEX4
TCEAL7
TCEAL9
BEX3
TCEAL4
TCEAL3
TCEAL1
MORF4L2-AS1
TMEM31
PLP1
ZCCHC18
FAM199X
IL1RAPL2
NRK
MUM1L1
CXorf57
RNF128
TBC1D8B
CLDN2
FRMPD3
PRPS1
MID2
VSIG1
ATG4A
COL4A5
NXT2
TMEM164
PAK3
SERTM2
ALG13
HTR2C
PLS3
AGTR2
SLC6A14
WDR44
DOCK11
IL13RA1
ZCCHC12
LINC01285
LONRF3
PGRMC1
AC004835.1
SLC25A43
SLC25A5
UBE2A
SOWAHD
NDUFA1
RHOXF1-AS1
ZBTB33
MCTS1
GLUD2
AL359851.1
GRIA3
XIAP
STAG2
SH2D1A
AL445072.1
OCRL
XPNPEP2
SASH3
UTP14A
BCORL1
RAB33A
SLC25A14
RBMX2
ARHGAP3

In [26]:
with open("gene.txt", "w") as txt:
    for gene in df['gene']:
        txt.write(str(gene)+'\n')

In [ ]:
dir = r'C:\Users\EcW78\CSE5713\Project\data'
fileset = os.listdir(dir)
for file in fileset:
    if file.endswith(".gz"):
        filepath = os.path.join(dir, file)
        print(filepath)
        print()
        df=pd.read_csv(gzip.open(os.path.join(dir, file)), sep='\t')
#         f_out = gzip.open(str(file))
#         with gzip.open(file) as f_in:
#             with open(file) as f_out:
#                 shutil.copyfileobj(f_in, f_out)
#         pass
#     elif file.endswith(".tsv"):
#         df=pd.read_csv(os.path.join(dir, file), sep='\t')
#         print(df)
    else:
        print("Other file type")

Other file type
C:\Users\EcW78\CSE5713\Project\data\GSM5075659_p007n.tsv.gz

Other file type
C:\Users\EcW78\CSE5713\Project\data\GSM5075660_p007t.tsv.gz

Other file type
C:\Users\EcW78\CSE5713\Project\data\GSM5075661_p008n.tsv.gz

Other file type
C:\Users\EcW78\CSE5713\Project\data\GSM5075662_p008t.tsv.gz

Other file type
C:\Users\EcW78\CSE5713\Project\data\GSM5075663_p009n1.tsv.gz

Other file type
C:\Users\EcW78\CSE5713\Project\data\GSM5075664_p009n2.tsv.gz

Other file type
C:\Users\EcW78\CSE5713\Project\data\GSM5075665_p009t1.tsv.gz

Other file type
C:\Users\EcW78\CSE5713\Project\data\GSM5075666_p009t2.tsv.gz

Other file type
C:\Users\EcW78\CSE5713\Project\data\GSM5075667_p012n.tsv.gz

